In [6]:
import os
import time
import numpy as np
import pandas as pd
from numpy import newaxis
from sklearn.model_selection import train_test_split
import glob
import matplotlib.pyplot as plt
import PIL

In [5]:
import tensorflow as tf

print(f"tensorflow version: {tf.__version__}")
print(f"tensorflow version: {tf.keras.__version__}")

tensorflow version: 2.5.0
tensorflow version: 2.5.0


In [34]:
# Import normal Dataset
def import_dataset(filename = "./Dataset/csv/Attacked/Original_X_test.csv"):
    with open(filename, newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        X_train, X_test = train_test_split(rows, test_size=0.2)
        X_train = np.array(X_train)
        X_test = np.array(X_test)
        X_train_ex = X_train[:, newaxis]
        X_test_ex = X_test[:, :, newaxis]
    return X_train, X_test, X_train_ex, X_test_ex

In [21]:
X_train, X_test = import_dataset()

In [48]:
from keras.models import Sequential

input_img = tf.keras.layers.Input(shape = (X_train.shape[1],))
encoder = tf.keras.layers.Dense(10, activation='relu')(input_img)
encoder = tf.keras.layers.Dense(8, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(2, activation='relu', name="latent_space")(encoder)

decoder = tf.keras.layers.Dense(8, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(10, activation='relu')(decoder)
decoder = tf.keras.layers.Dense(X_train.shape[1], activation='softmax')(decoder)

autoencoder = tf.keras.models.Model(input_img, decoder)
autoencoder.compile(loss='binary_crossentropy', optimizer= 'adam')

autoencoder.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense_40 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_41 (Dense)             (None, 8)                 88        
_________________________________________________________________
latent_space (Dense)         (None, 2)                 18        
_________________________________________________________________
dense_42 (Dense)             (None, 8)                 24        
_________________________________________________________________
dense_43 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_44 (Dense)             (None, 10)                110 

In [49]:
autoencoder.fit(X_train ,X_train ,epochs=10, batch_size=256,validation_data=(X_test, X_test))

Epoch 1/10
140/140 [==============================] - 1s 4ms/step - loss: -13.2453 - val_loss: -36.0155
Epoch 2/10
140/140 [==============================] - 0s 3ms/step - loss: -85.6157 - val_loss: -153.7113
Epoch 3/10
140/140 [==============================] - 0s 3ms/step - loss: -271.8967 - val_loss: -417.2727
Epoch 4/10
140/140 [==============================] - 0s 3ms/step - loss: -634.9857 - val_loss: -884.8526
Epoch 5/10
140/140 [==============================] - 0s 3ms/step - loss: -1229.6410 - val_loss: -1606.3561
Epoch 6/10
140/140 [==============================] - 0s 3ms/step - loss: -2102.0671 - val_loss: -2623.7695
Epoch 7/10
140/140 [==============================] - 0s 3ms/step - loss: -3291.8240 - val_loss: -3974.5588
Epoch 8/10
140/140 [==============================] - 0s 3ms/step - loss: -4832.2710 - val_loss: -5689.4482
Epoch 9/10
140/140 [==============================] - 1s 4ms/step - loss: -6751.4341 - val_loss: -7790.8706
Epoch 10/10
140/140 [==================

In [50]:
decode_msg = autoencoder.predict(X_test)

In [51]:
decode_msg[0]

array([0.0000000e+00, 1.0987715e-01, 5.8243882e-10, 8.9012164e-01,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.0672850e-13,
       1.2030154e-06, 0.0000000e+00], dtype=float32)

In [43]:
import sklearn
preds = autoencoder.predict(X_test)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(X_test, pred_labels[0:,1])

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Embedding, GlobalAveragePooling1D
epochs = 100
batch_size = 256

model = Sequential()
model.add(Conv1D(filters=100, kernel_size=5, padding='same',activation='relu',strides=1, input_shape=(X_train_ex.shape[1],1))) 
model.add(Dropout(0.2))
model.add(Conv1D(filters=200, kernel_size=5, padding='same', activation='relu',strides=1))
model.add(Dropout(0.2))
model.add(Conv1D(filters=400, kernel_size=10, padding='same', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.summary()